<a href="https://colab.research.google.com/github/Supriyasharma12/Carbon_emission_prediction/blob/main/Carbon_emission_prediction_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import feature_selection as fs
import numpy.random as nr

Data cleaning and preparation

In [ ]:
from google.colab import drive
import pandas as pd

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Set the full path to the project folder and Excel file
project_path = '/content/drive/MyDrive/Carbon_emission_project'
orig_data_file = f"{project_path}/CO2_emission_dataset.xls"
data_sheet = "Data"

# Step 3: Read the Excel file from the specified sheet
data_orig = pd.read_excel(io=orig_data_file, sheet_name=data_sheet)


Mounted at /content/drive


In [ ]:
print("Shape of the original dataset:")
data_orig.shape

Shape of the original dataset:


(13512, 28)

In [ ]:
print("Available columns:")
data_orig.columns

Available columns:


Index(['Country code', 'Country name',  'Series code',  'Series name',
              'SCALE',     'Decimals',           1990,           1991,
                 1992,           1993,           1994,           1995,
                 1996,           1997,           1998,           1999,
                 2000,           2001,           2002,           2003,
                 2004,           2005,           2006,           2007,
                 2008,           2009,           2010,           2011],
      dtype='object')

In [ ]:
print("Column data types:")
data_orig.dtypes

Column data types:


,0
Country code,object
Country name,object
Series code,object
Series name,object
SCALE,object
Decimals,object
1990,object
1991,object
1992,object
1993,object


In [ ]:
print("Overview of the first 5 rows:")
data_orig.head()

Overview of the first 5 rows:


,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.57481,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.967875,..,..,..,...,..,..,..,..,..,..,..,..,..,..


Data Cleaning and Conversion

In [ ]:
print("Descriptive statistics of the columns:")
data_orig.describe()

Descriptive statistics of the columns:


,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
count,13512,13512,13512,13512,13512,13512,10017,10017,10017,10017,...,10017,10017,10017,10017,10017,10017,10017,10017,10017,12382
unique,233,233,58,58,2,3,4355,3398,3523,3583,...,3877,3869,4007,4484,4008,4047,4080,3506,2164,1434
top,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
freq,58,58,233,233,10017,5823,5163,6520,6364,6300,...,5960,5974,5792,4933,5781,5769,5414,6256,7685,10244


In [ ]:
data_orig['Series name'].unique()

array(['Land area below 5m (% of land area)',
       'Agricultural land under irrigation (% of total ag. land)',
       'Cereal yield (kg per hectare)',
       'Foreign direct investment, net inflows (% of GDP)',
       'Access to electricity (% of total population)',
       'Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)',
       'Energy use per capita (kilograms of oil equivalent)',
       'CO2 emissions, total (KtCO2)',
       'CO2 emissions per capita (metric tons)',
       'CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)',
       'Other GHG emissions, total (KtCO2e)',
       'Methane (CH4) emissions, total (KtCO2e)',
       'Nitrous oxide (N2O) emissions, total (KtCO2e)',
       'Annex-I emissions reduction target',
       'Disaster risk reduction progress score (1-5 scale; 5=best)',
       'GHG net emissions/removals by LUCF (MtCO2e)',
       'Hosted Clean Development Mechanism (CDM) projects',
       'Hosted Joint Implementation (JI) projects',
       'Av

Select and Rename Relevant Variables

In [ ]:
data_orig['Series code'].unique()

array(['AG.LND.EL5M.ZS', 'AG.LND.IRIG.AG.ZS', 'AG.YLD.CREL.KG',
       'BX.KLT.DINV.WD.GD.ZS', 'EG.ELC.ACCS.ZS', 'EG.USE.COMM.GD.PP.KD',
       'EG.USE.PCAP.KG.OE', 'EN.ATM.CO2E.KT', 'EN.ATM.CO2E.PC',
       'EN.ATM.CO2E.PP.GD.KD', 'EN.ATM.GHGO.KT.CE', 'EN.ATM.METH.KT.CE',
       'EN.ATM.NOXE.KT.CE', 'EN.CLC.AERT', 'EN.CLC.DRSK.XQ',
       'EN.CLC.GHGR.MT.CE', 'EN.CLC.HCDM', 'EN.CLC.HJIP',
       'EN.CLC.HPPT.MM', 'EN.CLC.ICER', 'EN.CLC.IERU', 'EN.CLC.MDAT.ZS',
       'EN.CLC.MMDT.C', 'EN.CLC.NAMA', 'EN.CLC.NAPA', 'EN.CLC.NCOM',
       'EN.CLC.PCAT.C', 'EN.CLC.PCCC', 'EN.CLC.PCHW', 'EN.CLC.PCPT.MM',
       'EN.CLC.RNET', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS',
       'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ',
       'IE.PPI.ENGY.CD', 'IE.PPI.TELE.CD', 'IE.PPI.TRAN.CD',
       'IE.PPI.WATR.CD', 'IQ.CPA.PUBS.XQ', 'IS.ROD.PAVE.ZS',
       'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS',
       'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS',
       'SH.MED.NUMW.P3', 'S

In [ ]:
data_orig['SCALE'].unique()

array([0, 'Text'], dtype=object)

Handle Missing Values

In [ ]:
data_orig['Decimals'].unique()

array([1, 0, 'Text'], dtype=object)

In [ ]:
data_orig[data_orig['SCALE']=='Text']

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
3029,ABW,Aruba,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3030,ADO,Andorra,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3031,AFG,Afghanistan,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3032,AGO,Angola,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033,ALB,Albania,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7218,YEM,"Yemen, Rep.",EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,..
7219,ZAF,South Africa,EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
7220,ZAR,"Congo, Dem. Rep.",EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,..
7221,ZMB,Zambia,EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,..


In [ ]:
data_orig[data_orig['Decimals']=='Text']

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
3029,ABW,Aruba,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3030,ADO,Andorra,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3031,AFG,Afghanistan,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3032,AGO,Angola,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033,ALB,Albania,EN.CLC.AERT,Annex-I emissions reduction target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7218,YEM,"Yemen, Rep.",EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,..
7219,ZAF,South Africa,EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
7220,ZAR,"Congo, Dem. Rep.",EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,..
7221,ZMB,Zambia,EN.CLC.RNET,Renewable energy target,Text,Text,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,..


Export Cleaned Data

Removing rows marked as "Text" in the "SCALE" and "Decimals" columns


In [ ]:
# assign the data to a new DataFrame, which will be modified
data_clean = data_orig

print("Original number of rows:")
print(data_clean.shape[0])

# remove rows characterized as "Text" in the SCALE column
data_clean = data_clean[data_clean['SCALE']!='Text']

print("Current number of rows:")
print(data_clean.shape[0])

Original number of rows:
13512
Current number of rows:
10017


Removing the unnecessary columns "Country name", "Series code", "SCALE", "Decimals"

In [ ]:
print("Original number of columns:")
print(data_clean.shape[1])

data_clean = data_clean.drop(['Country name', 'Series code', 'SCALE', 'Decimals'], axis='columns')

print("Current number of columns:")
print(data_clean.shape[1])

Original number of columns:
28
Current number of columns:
24


Transform the ".." strings and emplty cells ("") into NaN values for easier recognission as missing values

In [ ]:
data_clean.iloc[:,2:] = data_clean.iloc[:,2:].replace({'':np.nan, '..':np.nan})

/tmp/ipython-input-17-1081394343.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_clean.iloc[:,2:] = data_clean.iloc[:,2:].replace({'':np.nan, '..':np.nan})


Transform all data columns into a numerical data type

In [ ]:
data_clean2 = data_orig.copy()

# Step 2: (Optional) Perform your cleaning/transformation
# Example: Drop rows with missing values
data_clean2 = data_clean2.dropna()

# Step 3: Now print the column data types
print("Print the column data types after transformation:")
print(data_clean2.dtypes)

Print the column data types after transformation:
Country code    object
Country name    object
Series code     object
Series name     object
SCALE           object
Decimals        object
1990            object
1991            object
1992            object
1993            object
1994            object
1995            object
1996            object
1997            object
1998            object
1999            object
2000            object
2001            object
2002            object
2003            object
2004            object
2005            object
2006            object
2007            object
2008            object
2009            object
2010            object
2011            object
dtype: object


In [ ]:
# define shorter names corresponding to most relevant variables in a dictionary
chosen_vars = {'Cereal yield (kg per hectare)': 'cereal_yield',
               'Foreign direct investment, net inflows (% of GDP)': 'fdi_perc_gdp',
               'Access to electricity (% of total population)': 'elec_access_perc',
               'Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)': 'en_per_gdp',
               'Energy use per capita (kilograms of oil equivalent)': 'en_per_cap',
               'CO2 emissions, total (KtCO2)': 'co2_ttl',
               'CO2 emissions per capita (metric tons)': 'co2_per_cap',
               'CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)': 'co2_per_gdp',
               'Other GHG emissions, total (KtCO2e)': 'other_ghg_ttl',
               'Methane (CH4) emissions, total (KtCO2e)': 'ch4_ttl',
               'Nitrous oxide (N2O) emissions, total (KtCO2e)': 'n2o_ttl',
               'Droughts, floods, extreme temps (% pop. avg. 1990-2009)': 'nat_emerg',
               'Population in urban agglomerations >1million (%)': 'pop_urb_aggl_perc',
               'Nationally terrestrial protected areas (% of total land area)': 'prot_area_perc',
               'GDP ($)': 'gdp',
               'GNI per capita (Atlas $)': 'gni_per_cap',
               'Under-five mortality rate (per 1,000)': 'under_5_mort_rate',
               'Population growth (annual %)': 'pop_growth_perc',
               'Population': 'pop',
               'Urban population growth (annual %)': 'urb_pop_growth_perc',
               'Urban population': 'urb_pop'
                }

# rename all variables in the column "Series name" with comprehensible shorter versions
data_clean2['Series name'] = data_clean2['Series name'].replace(to_replace=chosen_vars)

In [ ]:
data_clean2.head()

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.57481,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.967875,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [ ]:
# save the short feature names into a list of strings
chosen_cols = list(chosen_vars.values())

# define an empty list, where sub-dataframes for each feature will be saved
frame_list = []

# iterate over all chosen features
for variable in chosen_cols:

    # pick only rows corresponding to the current feature
    frame = data_clean2[data_clean2['Series name'] == variable]

    # melt all the values for all years into one column and rename the columns correspondingly
    frame = frame.melt(id_vars=['Country code', 'Series name']).rename(columns={'Country code': 'country', 'variable': 'year', 'value': variable}).drop(['Series name'], axis='columns')

    # add the melted dataframe for the current feature into the list
    frame_list.append(frame)


# merge all sub-frames into a single dataframe, making an outer binding on the key columns 'country','year'
from functools import reduce
all_vars = reduce(lambda left, right: pd.merge(left, right, on=['country','year'], how='outer'), frame_list)

In [ ]:
all_vars.head()

,country,year,cereal_yield,fdi_perc_gdp,elec_access_perc,en_per_gdp,en_per_cap,co2_ttl,co2_per_cap,co2_per_gdp,...,nat_emerg,pop_urb_aggl_perc,prot_area_perc,gdp,gni_per_cap,under_5_mort_rate,pop_growth_perc,pop,urb_pop_growth_perc,urb_pop
0,ABW,1990,..,..,..,..,..,1840.834,29.620641,..,...,..,..,0.105547,..,..,..,1.820254,62147,1.780501,31259.941
1,ABW,1991,..,21.185138,..,..,..,1928.842,29.838835,..,...,..,..,0.10493,872067021.913829,..,..,3.936181,64642,3.337974,32321
2,ABW,1992,..,-3.857809,..,..,..,1723.49,25.239288,..,...,..,..,0.10493,958659238.282076,..,..,5.484041,68286,4.882234,33938.142
3,ABW,1993,..,-1.655492,..,..,..,1771.161,24.405233,..,...,..,..,0.10493,1083240282.27456,14940,..,6.088818,72573,5.483368,35851.062
4,ABW,1994,..,-5.874439,..,..,..,1763.827,22.977867,..,...,..,..,0.10493,1245810153.89127,15320,..,5.611677,76762,5.002538,37690.142


Remove the remaining missing values in an optimal way

In [ ]:
print("check the amount of missing values in each column")
all_vars.isnull().sum()

check the amount of missing values in each column


,0
country,0
year,0
cereal_yield,0
fdi_perc_gdp,0
elec_access_perc,0
en_per_gdp,0
en_per_cap,0
co2_ttl,0
co2_per_cap,0
co2_per_gdp,0


Handling Missing Values

Filtering the years by missing values

In [ ]:
all_vars_clean = all_vars

#define an array with the unique year values
years_count_missing = dict.fromkeys(all_vars_clean['year'].unique(), 0)
for ind, row in all_vars_clean.iterrows():
    years_count_missing[row['year']] += row.isnull().sum()

# sort the years by missing values
years_missing_sorted = dict(sorted(years_count_missing.items(), key=lambda item: item[1]))

# print the missing values for each year
print("missing values by year:")
for key, val in years_missing_sorted.items():
    print(key, ":", val)

missing values by year:
1990 : 2
1991 : 2
1992 : 2
1993 : 2
1994 : 2
1995 : 2
1996 : 2
1997 : 2
1998 : 2
1999 : 2
2000 : 2
2001 : 2
2002 : 2
2003 : 2
2004 : 2
2005 : 2
2006 : 2
2007 : 2
2008 : 2
2009 : 2
2010 : 2
2011 : 2
Country name : 2
Decimals : 2
SCALE : 2
Series code : 2


Filtering by Year

In [ ]:
# print("number of missing values in the whole dataset before filtering the years:")
# print(all_vars_clean.isnull().sum().sum())
# print("number of rows before filtering the years:")
# print(all_vars_clean.shape[0])

# # filter only rows for years between 1991 and 2008 (having less missing values)
# all_vars_clean = all_vars_clean[(all_vars_clean['year'] >= 1991) & (all_vars_clean['year'] <= 2008)]

# print("number of missing values in the whole dataset after filtering the years:")
# print(all_vars_clean.isnull().sum().sum())
# print("number of rows after filtering the years:")
# print(all_vars_clean.shape[0])


# Step 1: Convert 'year' column to numeric (very important!)
all_vars_clean['year'] = pd.to_numeric(all_vars_clean['year'], errors='coerce')

# Step 2: Print info before filtering
print("number of missing values in the whole dataset before filtering the years:")
print(all_vars_clean.isnull().sum().sum())

print("number of rows before filtering the years:")
print(all_vars_clean.shape[0])

# Step 3: Filter only rows where year is between 1991 and 2008
all_vars_clean = all_vars_clean[
    (all_vars_clean['year'] >= 1991) & (all_vars_clean['year'] <= 2008)
]

# Step 4: Print info after filtering
print("number of missing values in the whole dataset after filtering the years:")
print(all_vars_clean.isnull().sum().sum())

print("number of rows after filtering the years:")
print(all_vars_clean.shape[0])


number of missing values in the whole dataset before filtering the years:
984
number of rows before filtering the years:
6058
number of missing values in the whole dataset after filtering the years:
36
number of rows after filtering the years:
4194


Filtering the countries by missing values

In [ ]:
# check the amount of missing values by country

# define an array with the unique country values
countries_count_missing = dict.fromkeys(all_vars_clean['country'].unique(), 0)

# iterate through all rows and count the amount of NaN values for each country
for ind, row in all_vars_clean.iterrows():
    countries_count_missing[row['country']] += row.isnull().sum()

# sort the countries by missing values
countries_missing_sorted = dict(sorted(countries_count_missing.items(), key=lambda item: item[1]))

# print the missing values for each country
print("missing values by country:")
for key, val in countries_missing_sorted.items():
    print(key, ":", val)

missing values by country:
ABW : 0
ADO : 0
AFG : 0
AGO : 0
ALB : 0
ARE : 0
ARG : 0
ARM : 0
ASM : 0
ATG : 0
AUS : 0
AUT : 0
AZE : 0
BDI : 0
BEL : 0
BEN : 0
BFA : 0
BGD : 0
BGR : 0
BHR : 0
BHS : 0
BIH : 0
BLR : 0
BLZ : 0
BMU : 0
BOL : 0
BRA : 0
BRB : 0
BRN : 0
BTN : 0
BWA : 0
CAF : 0
CAN : 0
CHE : 0
CHI : 0
CHL : 0
CHN : 0
CIV : 0
CMR : 0
COG : 0
COK : 0
COL : 0
COM : 0
CPV : 0
CRI : 0
CUB : 0
CUW : 0
CYM : 0
CYP : 0
CZE : 0
DEU : 0
DJI : 0
DMA : 0
DNK : 0
DOM : 0
DZA : 0
EAP : 0
ECA : 0
ECU : 0
EGY : 0
EMU : 0
ERI : 0
ESP : 0
EST : 0
ETH : 0
FIN : 0
FJI : 0
FRA : 0
FRO : 0
FSM : 0
GAB : 0
GBR : 0
GEO : 0
GHA : 0
GIB : 0
GIN : 0
GMB : 0
GNB : 0
GNQ : 0
GRC : 0
GRD : 0
GRL : 0
GTM : 0
GUM : 0
GUY : 0
HIC : 0
HKG : 0
HND : 0
HRV : 0
HTI : 0
HUN : 0
IDN : 0
IMY : 0
IND : 0
IRL : 0
IRN : 0
IRQ : 0
ISL : 0
ISR : 0
ITA : 0
JAM : 0
JOR : 0
JPN : 0
KAZ : 0
KEN : 0
KGZ : 0
KHM : 0
KIR : 0
KNA : 0
KOR : 0
KSV : 0
KWT : 0
LAC : 0
LAO : 0
LBN : 0
LBR : 0
LBY : 0
LCA : 0
LIC : 0
LIE : 0
LKA : 0
LMC :

In [ ]:
print("number of missing values in the whole dataset before filtering the countries:")
print(all_vars_clean.isnull().sum().sum())
print("number of rows before filtering the countries:")
print(all_vars_clean.shape[0])


# filter only rows for countries with less than 90 missing values
countries_filter = []
for key, val in countries_missing_sorted.items():
    if val<90:
        countries_filter.append(key)

all_vars_clean = all_vars_clean[all_vars_clean['country'].isin(countries_filter)]

print("number of missing values in the whole dataset after filtering the countries:")
print(all_vars_clean.isnull().sum().sum())
print("number of rows after filtering the countries:")
print(all_vars_clean.shape[0])

number of missing values in the whole dataset before filtering the countries:
36
number of rows before filtering the countries:
4194
number of missing values in the whole dataset after filtering the countries:
36
number of rows after filtering the countries:
4194


Checking the features (columns) for missing values

In [ ]:
print("number of missing values in the whole dataset before filtering the countries:")
print(all_vars_clean.isnull().sum().sum())
print("number of rows before filtering the countries:")
print(all_vars_clean.shape[0])


# filter only rows for countries with less than 90 missing values
countries_filter = []
for key, val in countries_missing_sorted.items():
    if val<90:
        countries_filter.append(key)

all_vars_clean = all_vars_clean[all_vars_clean['country'].isin(countries_filter)]

print("number of missing values in the whole dataset after filtering the countries:")
print(all_vars_clean.isnull().sum().sum())
print("number of rows after filtering the countries:")
print(all_vars_clean.shape[0])

number of missing values in the whole dataset before filtering the countries:
36
number of rows before filtering the countries:
4194
number of missing values in the whole dataset after filtering the countries:
36
number of rows after filtering the countries:
4194


In [ ]:
all_vars_clean.isnull().sum()

,0
country,0
year,0
cereal_yield,0
fdi_perc_gdp,0
elec_access_perc,0
en_per_gdp,0
en_per_cap,0
co2_ttl,0
co2_per_cap,0
co2_per_gdp,0


Dropping High-NaN Features

In [ ]:
# remove features with more than 20 missing values

from itertools import compress

# create a boolean mapping of features with more than 20 missing values
vars_bad = all_vars_clean.isnull().sum()>20

# remove the columns corresponding to the mapping of the features with many missing values
all_vars_clean2 = all_vars_clean.drop(compress(data = all_vars_clean.columns, selectors = vars_bad), axis='columns')

print("Remaining missing values per column:")
print(all_vars_clean2.isnull().sum())

Remaining missing values per column:
country                 0
year                    0
cereal_yield            0
fdi_perc_gdp            0
elec_access_perc        0
en_per_gdp              0
en_per_cap              0
co2_ttl                 0
co2_per_cap             0
co2_per_gdp             0
other_ghg_ttl           0
ch4_ttl                 0
n2o_ttl                 0
nat_emerg               0
pop_urb_aggl_perc       0
prot_area_perc          0
gdp                     0
gni_per_cap             0
under_5_mort_rate       0
pop_growth_perc        18
pop                     0
urb_pop_growth_perc    18
urb_pop                 0
dtype: int64


In [ ]:
# delete rows with any number of missing values
all_vars_clean3 = all_vars_clean2.dropna(axis='rows', how='any')

print("Remaining missing values per column:")
print(all_vars_clean3.isnull().sum())

print("Final shape of the cleaned dataset:")
print(all_vars_clean3.shape)

Remaining missing values per column:
country                0
year                   0
cereal_yield           0
fdi_perc_gdp           0
elec_access_perc       0
en_per_gdp             0
en_per_cap             0
co2_ttl                0
co2_per_cap            0
co2_per_gdp            0
other_ghg_ttl          0
ch4_ttl                0
n2o_ttl                0
nat_emerg              0
pop_urb_aggl_perc      0
prot_area_perc         0
gdp                    0
gni_per_cap            0
under_5_mort_rate      0
pop_growth_perc        0
pop                    0
urb_pop_growth_perc    0
urb_pop                0
dtype: int64
Final shape of the cleaned dataset:
(4176, 23)


Export of the cleaned data frame to a file

In [ ]:
# Replace this path with your desired folder in Drive
output_path = '/content/drive/MyDrive/Carbon_emission_project/data_cleaned.csv'

# Save the CSV file
all_vars_clean3.to_csv(output_path, index=False)

print("File saved to Drive:", output_path)


File saved to Drive: /content/drive/MyDrive/Carbon_emission_project/data_cleaned.csv
